In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras import Model
import tensorflow.keras.backend as K
import tensorflow_addons as tfa

Create dummy data

In [ ]:
x_0 = np.random.uniform(0, 1, size=(100, 1024))
x_1 = x = np.random.uniform(0, 1, size=(100, 1024))
y = np.random.randint(2, size=(100))

Create Siamese model

In [ ]:
# Branch
K.clear_session()
i = Input(shape=(1024,))
embd = Dense(128, activation='relu')(i)
branch = Model(i, embd, name='branch')

# Siamese network
i_0 = Input(shape=(1024,))
i_1 = Input(shape=(1024,))
embd_0 = branch(i_0)
embd_1 = branch(i_1)
dist = Lambda(lambda a: tf.norm(a[0]-a[1], ord='euclidean', axis=1))([embd_0, embd_1])
siamese = Model([i_0, i_1], dist, name='siamese')

# Summary
print(branch.summary())
print(siamese.summary())

In [ ]:
def loss(y_true, y_pred):
    #y_true = tf.squeeze(y_true)
    loss = tfa.losses.ContrastiveLoss()(y_true, y_pred)
    return loss

siamese.compile(
    optimizer='sgd',
    loss=loss,
    run_eagerly=False
)

Train model

In [ ]:
siamese.fit(x=(x_0, x_1), y=y, batch_size=10, epochs=10)

In [ ]:
a = tf.constant(np.array([1, 2, 3], dtype=int))
b = tf.constant(np.array([0.1, 0.2, 0.3], dtype=float))
tfa.losses.ContrastiveLoss()(a, b)

In [ ]:
y_pred = siamese.predict((x_0, x_1))

In [ ]:
# Average embeddings distances when 1) y = 0, 2) y = 1
a = y_pred[np.where(y==0)[0]]
print(np.min(a), np.mean(a), np.max(a))
b = y_pred[np.where(y==1)[0]]
print(np.min(b), np.mean(b), np.max(b))